Dependencies

In [1]:
!pip install geopandas
!pip install shapely

Un/zip Files

In [2]:
!unzip mean-sea-level-pressure-2020.zip

Archive:  mean-sea-level-pressure-2020.zip
  inflating: data_stream-oper.nc     


In [ ]:
!zip -r /content/merged.zip /content/merged

Processing ERA5 to create polygons with attributes

In [7]:
import xarray as xr

filename = 'data_stream-oper.nc'

# Load the NetCDF dataset
ds = xr.open_dataset(filename)
ds

<xarray.Dataset> Size: 430MB
Dimensions:     (valid_time: 856, latitude: 261, longitude: 481)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 7kB 2020-05-01 ... 2020-11-30T18:...
  * latitude    (latitude) float64 2kB 65.0 64.75 64.5 64.25 ... 0.5 0.25 0.0
  * longitude   (longitude) float64 4kB -120.0 -119.8 -119.5 ... -0.5 -0.25 0.0
    expver      (valid_time) <U4 14kB ...
Data variables:
    msl         (valid_time, latitude, longitude) float32 430MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-19T23:30 GRIB to CDM+CF via cfgrib-0.9.1...

In [11]:
# Remove the attribute dimension
#ds = ds.squeeze(dim='msl')

# Round latitude and longitude to the nearest degree
ds = ds.assign_coords(
    lat_rounded=ds.latitude.round(0),
    lon_rounded=ds.longitude.round(0)
)
# Group by the rounded coordinates and compute the mean temperature
ds_grouped = ds.groupby(['lat_rounded', 'lon_rounded']).mean()
print(ds_grouped)

<xarray.Dataset> Size: 708B
Dimensions:      (lat_rounded: 5, lon_rounded: 5, valid_time: 5)
Coordinates:
  * lat_rounded  (lat_rounded) float64 40B 0.0 1.0 2.0 3.0 4.0
  * lon_rounded  (lon_rounded) float64 40B -120.0 -119.0 -118.0 -117.0 -116.0
    number       int64 8B 0
  * valid_time   (valid_time) datetime64[ns] 40B 2020-05-01 ... 2020-05-02
    expver       (valid_time) <U4 80B '0001' '0001' '0001' '0001' '0001'
Data variables:
    msl          (valid_time, lat_rounded, lon_rounded) float32 500B 1.009e+0...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-19T23:30 GRIB to CDM+CF via cfgrib-0.9.1...


In [14]:
# Convert the grouped dataset to a DataFrame for tabular display
df_grouped = ds_grouped.to_dataframe().reset_index()

# Print the resulting DataFrame
print(df_grouped[:10])

   lat_rounded  lon_rounded          valid_time            msl  number expver
0          0.0       -120.0 2020-05-01 00:00:00  100877.593750       0   0001
1          0.0       -120.0 2020-05-01 06:00:00  101223.148438       0   0001
2          0.0       -120.0 2020-05-01 12:00:00  100960.304688       0   0001
3          0.0       -120.0 2020-05-01 18:00:00  101278.570312       0   0001
4          0.0       -120.0 2020-05-02 00:00:00  100917.718750       0   0001
5          0.0       -120.0 2020-05-02 06:00:00  101290.343750       0   0001
6          0.0       -120.0 2020-05-02 12:00:00  101031.335938       0   0001
7          0.0       -120.0 2020-05-02 18:00:00  101348.242188       0   0001
8          0.0       -120.0 2020-05-03 00:00:00  100988.062500       0   0001
9          0.0       -120.0 2020-05-03 06:00:00  101294.656250       0   0001


In [20]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
from datetime import datetime
from typing import Union, List, Optional, Dict

def create_grid_cell_polygon(longitude: float, latitude: float, cell_size: float = 1.0) -> Polygon:
    """
    Create a square grid cell centered at (longitude, latitude) with given cell size.

    Parameters:
        longitude (float): Center point longitude
        latitude (float): Center point latitude
        cell_size (float): Size of the grid cell in degrees

    Returns:
        Polygon: Square polygon representing the grid cell
    """
    half_size = cell_size / 2
    return Polygon([
        (longitude - half_size, latitude - half_size),  # bottom left
        (longitude - half_size, latitude + half_size),  # top left
        (longitude + half_size, latitude + half_size),  # top right
        (longitude + half_size, latitude - half_size),  # bottom right
        (longitude - half_size, latitude - half_size)   # back to start
    ])

def process_era5_grid(
    era5_data: pd.DataFrame,
    time_column: str = 'valid_time',
    lat_column: str = 'latitude',
    lon_column: str = 'longitude',
    value_column: str = 'msl',
    grid_cell_size: float = 1.0,
    simplify_tolerance: Optional[float] = 0.01,
    round_decimals: Optional[int] = 1,
    output_filename_pattern: str = 'msl_{}.geojson',
    crs: str = "EPSG:4326"
)  -> Dict[datetime, Dict]:
    """
    Convert ERA5 gridded data into GeoJSON files, one for each timestamp.

    Parameters:
        era5_data (pd.DataFrame): ERA5 data with columns for time, lat, lon, and values
        time_column (str): Name of timestamp column
        lat_column (str): Name of latitude column
        lon_column (str): Name of longitude column
        value_column (str): Name of data value column (e.g., temperature, pressure)
        grid_cell_size (float): Size of each grid cell in degrees
        simplify_tolerance (float): Tolerance for simplifying polygon geometries
        round_decimals (int): Number of decimal places to round values
        output_filename_pattern (str): Pattern for output files (e.g., 'temperature_{}.geojson')
        crs (str): Coordinate reference system
     Returns:
        Dict[str, Dict]: Dictionary mapping timestamp strings to GeoJSON objects
    """
    # Initialize results dictionary
    geojson_results = {}

    # Ensure timestamp column is datetime
    if not pd.api.types.is_datetime64_any_dtype(era5_data[time_column]):
        era5_data[time_column] = pd.to_datetime(era5_data[time_column])

    # Get list of unique timestamps
    unique_timestamps = sorted(era5_data[time_column].unique())

    # Process each timestamp separately
    for timestamp in unique_timestamps:
        print(f"Processing data for {timestamp}...")

        # Extract data for this timestamp
        current_timestamp_data = era5_data[era5_data[time_column] == timestamp].copy()

        # Get unique coordinate pairs for this timestamp
        unique_coordinates = current_timestamp_data[[lat_column, lon_column]].drop_duplicates()

        # Initialize lists to store our processed data
        grid_cells = []         # Will store the polygon for each grid cell
        grid_values = []        # Will store the data value for each cell
        grid_latitudes = []     # Will store the latitude of each cell
        grid_longitudes = []    # Will store the longitude of each cell

        # Process each unique coordinate pair
        total_coords = len(unique_coordinates)
        for idx, coordinate in unique_coordinates.iterrows():
            # Extract the latitude and longitude
            latitude = coordinate[lat_column]
            longitude = coordinate[lon_column]

            # Find the matching data value for this location
            matching_data = current_timestamp_data[
                (current_timestamp_data[lat_column] == latitude) &
                (current_timestamp_data[lon_column] == longitude)
            ]

            # Get the data value (temperature, pressure, etc.)
            data_value = matching_data[value_column].iloc[0]

            # Skip if the value is missing
            if pd.isna(data_value):
                continue

            # Create a polygon representing this grid cell
            grid_cell = create_grid_cell_polygon(
                longitude=longitude,
                latitude=latitude,
                cell_size=grid_cell_size
            )

            # Store all information for this grid cell
            grid_cells.append(grid_cell)
            grid_values.append(data_value)
            grid_latitudes.append(latitude)
            grid_longitudes.append(longitude)


        # Create GeoDataFrame from processed data
        geodata = gpd.GeoDataFrame(
            {
                time_column: timestamp,
                value_column: grid_values,
                lat_column: grid_latitudes,
                lon_column: grid_longitudes,
                'geometry': grid_cells
            },
            crs=crs
        )

        # Simplify the polygons if requested
        if simplify_tolerance is not None:
            geodata['geometry'] = geodata['geometry'].simplify(
                tolerance=simplify_tolerance,
                preserve_topology=True
            )

        # Round the data values if requested
        if round_decimals is not None:
            geodata[value_column] = geodata[value_column].round(round_decimals)

        # Convert to GeoJSON
        geojson_data = geodata.__geo_interface__

        # Store in results dictionary
        geojson_results[timestamp] = geojson_data

    return geojson_results

In [21]:
geojson_results = process_era5_grid(
    era5_data=df_grouped,
    time_column='valid_time',
    lat_column='lat_rounded',
    lon_column='lon_rounded',
    value_column='msl',
    grid_cell_size=1.0,
    round_decimals=1,
    simplify_tolerance = 0.01,
    output_filename_pattern = 'msl/msl_{}.geojson',
    crs="EPSG:4326"
)

geojson_results

Processing data for 2020-05-01 00:00:00...
Processing data for 2020-05-01 06:00:00...
Processing data for 2020-05-01 12:00:00...
Processing data for 2020-05-01 18:00:00...
Processing data for 2020-05-02 00:00:00...
Processing data for 2020-05-02 06:00:00...
Processing data for 2020-05-02 12:00:00...
Processing data for 2020-05-02 18:00:00...
Processing data for 2020-05-03 00:00:00...
Processing data for 2020-05-03 06:00:00...
Processing data for 2020-05-03 12:00:00...
Processing data for 2020-05-03 18:00:00...
Processing data for 2020-05-04 00:00:00...
Processing data for 2020-05-04 06:00:00...
Processing data for 2020-05-04 12:00:00...
Processing data for 2020-05-04 18:00:00...
Processing data for 2020-05-05 00:00:00...
Processing data for 2020-05-05 06:00:00...
Processing data for 2020-05-05 12:00:00...
Processing data for 2020-05-05 18:00:00...
Processing data for 2020-05-06 00:00:00...
Processing data for 2020-05-06 06:00:00...
Processing data for 2020-05-06 12:00:00...
Processing 

KeyboardInterrupt: 

In [22]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np

def process_era5_grid_optimized(
    era5_data: pd.DataFrame,
    time_column: str = 'valid_time',
    lat_column: str = 'latitude',
    lon_column: str = 'longitude',
    value_column: str = 'msl',
    grid_cell_size: float = 1.0,
    simplify_tolerance: Optional[float] = 0.01,
    round_decimals: Optional[int] = 1,
    crs: str = "EPSG:4326"
) -> Dict[datetime, Dict]:
    """
    Optimized version of process_era5_grid.
    """
    geojson_results = {}

    # Ensure timestamp column is datetime
    era5_data[time_column] = pd.to_datetime(era5_data[time_column])
    unique_timestamps = era5_data[time_column].unique()

    # Pivot data for efficient lookup
    pivoted_data = era5_data.pivot_table(
        index=[lat_column, lon_column],
        columns=time_column,
        values=value_column
    ).dropna().reset_index()

    # Precompute grid cells for unique lat/lon pairs
    def create_polygon(row):
        lat, lon = row[lat_column], row[lon_column]
        half_size = grid_cell_size / 2
        return Polygon([
            (lon - half_size, lat - half_size),
            (lon - half_size, lat + half_size),
            (lon + half_size, lat + half_size),
            (lon + half_size, lat - half_size),
            (lon - half_size, lat - half_size)
        ])

    pivoted_data['geometry'] = pivoted_data.apply(create_polygon, axis=1)

    # Process each timestamp
    for timestamp in unique_timestamps:
        print(f"Processing data for {timestamp}...")

        # Get data for this timestamp
        timestamp_data = pivoted_data[[lat_column, lon_column, 'geometry', timestamp]].dropna()
        timestamp_data.columns = [lat_column, lon_column, 'geometry', value_column]

        # Round values if requested
        if round_decimals is not None:
            timestamp_data[value_column] = timestamp_data[value_column].round(round_decimals)

        # Convert to GeoDataFrame
        geodata = gpd.GeoDataFrame(timestamp_data, crs=crs)

        # Simplify if needed
        if simplify_tolerance:
            geodata['geometry'] = geodata['geometry'].simplify(tolerance=simplify_tolerance)

        # Convert to GeoJSON and store
        geojson_results[timestamp] = geodata.__geo_interface__

    return geojson_results


In [ ]:
geojson_results = process_era5_grid_optimized(
    era5_data=df_grouped,
    time_column='valid_time',
    lat_column='lat_rounded',
    lon_column='lon_rounded',
    value_column='msl',
    grid_cell_size=1.0,
    round_decimals=1,
    simplify_tolerance = 0.01,
    crs="EPSG:4326"
)

geojson_results

Processing data for 2020-05-01 00:00:00...
Processing data for 2020-05-01 06:00:00...
Processing data for 2020-05-01 12:00:00...
Processing data for 2020-05-01 18:00:00...
Processing data for 2020-05-02 00:00:00...
Processing data for 2020-05-02 06:00:00...
Processing data for 2020-05-02 12:00:00...
Processing data for 2020-05-02 18:00:00...
Processing data for 2020-05-03 00:00:00...
Processing data for 2020-05-03 06:00:00...
Processing data for 2020-05-03 12:00:00...
Processing data for 2020-05-03 18:00:00...
Processing data for 2020-05-04 00:00:00...
Processing data for 2020-05-04 06:00:00...
Processing data for 2020-05-04 12:00:00...
Processing data for 2020-05-04 18:00:00...
Processing data for 2020-05-05 00:00:00...
Processing data for 2020-05-05 06:00:00...
Processing data for 2020-05-05 12:00:00...
Processing data for 2020-05-05 18:00:00...
Processing data for 2020-05-06 00:00:00...
Processing data for 2020-05-06 06:00:00...
Processing data for 2020-05-06 12:00:00...
Processing 

Reading geojson files to merge polygons and reduce file size

In [ ]:
import json
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import numpy as np

def kelvin_to_celsius(kelvin):
    return float(kelvin) - 273.15

def get_temp_bucket(temp_celsius):
    """
    Assigns temperature to a fixed 0.5°C bucket
    Example: 20.3°C -> 20.0-20.5 bucket, 20.7°C -> 20.5-21.0 bucket
    Returns the lower bound of the bucket
    """
    return np.floor(temp_celsius * 2) / 2

def are_polygons_adjacent(geom1, geom2):
    """Check if two polygons are touching"""
    return geom1.touches(geom2) or geom1.intersects(geom2)

def merge_temperature_ranges(geojson_data, debug=True):
    features = geojson_data['features']

    # First, group features by temperature buckets
    temp_buckets = {}
    for feature in features:
        temp = kelvin_to_celsius(float(feature['properties']['temperature']))
        bucket = get_temp_bucket(temp)
        if bucket not in temp_buckets:
            temp_buckets[bucket] = []
        temp_buckets[bucket].append(feature)

    if debug:
        print("\nTemperature buckets:")
        for bucket in sorted(temp_buckets.keys()):
            print(f"{bucket}°C to {bucket+0.5}°C: {len(temp_buckets[bucket])} polygons")

    merged_features = []

    # Process each temperature bucket
    for bucket_temp in sorted(temp_buckets.keys()):
        bucket_features = temp_buckets[bucket_temp]
        if debug:
            print(f"\nProcessing bucket {bucket_temp}°C to {bucket_temp+0.5}°C")

        # Keep track of which features in this bucket we've processed
        processed = set()

        # Find connected groups within each temperature bucket
        for i, feature in enumerate(bucket_features):
            if i in processed:
                continue

            current_geom = shape(feature['geometry'])
            current_group = [feature]
            processed.add(i)

            # Find all adjacent polygons in the same temperature bucket
            changed = True
            while changed:
                changed = False
                for j, other_feature in enumerate(bucket_features):
                    if j in processed:
                        continue

                    other_geom = shape(other_feature['geometry'])
                    if are_polygons_adjacent(current_geom, other_geom):
                        current_group.append(other_feature)
                        current_geom = unary_union([current_geom, other_geom])
                        processed.add(j)
                        changed = True

            # Create merged feature for this connected group
            temps_kelvin = [float(f['properties']['temperature']) for f in current_group]
            avg_temp_kelvin = np.mean(temps_kelvin)

            if debug:
                print(f"  Created group of {len(current_group)} polygons")

            merged_feature = {
                "type": "Feature",
                "properties": {
                    "valid_time": current_group[0]['properties']['valid_time'],
                    "temperature": str(kelvin_to_celsius(avg_temp_kelvin)),
                    "latitude": current_geom.centroid.y,
                    "longitude": current_geom.centroid.x,
                    "merged_count": len(current_group),
                    "temp_bucket": f"{bucket_temp:.1f}°C to {bucket_temp+0.5:.1f}°C",
                    "lower_bound_temp": f"{bucket_temp:.1f}",
                    "upper_bound_temp": f"{bucket_temp+0.5:.1f}",
                },
                "geometry": mapping(current_geom)
            }
            merged_features.append(merged_feature)

    output_geojson = {
        "type": "FeatureCollection",
        "name": geojson_data['name'],
        "crs": geojson_data['crs'],
        "features": merged_features
    }

    return output_geojson

def process_geojson_file(input_file, output_file, debug=True):
    """
    Process a GeoJSON file and save the merged result

    Args:
        input_file (str): Path to input GeoJSON file
        output_file (str): Path to save merged GeoJSON
        debug (bool): Whether to print debug information
    """
    with open(input_file, 'r') as f:
        geojson_data = json.load(f)

    merged_geojson = merge_temperature_ranges(geojson_data, debug)

    with open(output_file, 'w') as f:
        json.dump(merged_geojson, f)

    # Print reduction statistics
    original_features = len(geojson_data['features'])
    merged_features = len(merged_geojson['features'])
    reduction = (1 - merged_features/original_features) * 100

    print(f"\nSummary:")
    print(f"Original features: {original_features}")
    print(f"Merged features: {merged_features}")
    print(f"Reduction: {reduction:.1f}%")
    print(f"Output File: {output_file}")

# Example usage
# process_geojson_file('temperature_20200101_000000.geojson',
#                         'temperature_20200101_000000_merged.geojson')

In [ ]:
# prompt: generate a date object for jan 1 2020 00:00 and print out date like this 20200101_000000

from datetime import datetime, timedelta

date_object = datetime(2020, 5, 16, 0, 0)


while date_object != datetime(2021, 5, 21, 0, 0):
    formatted_input = date_object.strftime('content/processed/temperature_%Y%m%d_%H%M%S.geojson')
    formatted_output = date_object.strftime('merged/temperature_%Y%m%d_%H%M%S_merged.geojson')
    process_geojson_file(formatted_input, formatted_output, False)
    date_object = date_object + timedelta(hours=6)